In [2]:
import os
import sys
import glob
import pysam
import gzip
import subprocess
import datetime
import tqdm
from Bio import SeqIO, bgzf
from Bio.Seq import Seq, reverse_complement
from Bio.SeqRecord import SeqRecord

import csv
import argparse

import pandas as pd
import numpy as np
from multiprocessing import Process

In [26]:
class Preprocess:
    def __init__(self, input_file, output_file, tmp_path, adapter, sample_name, flag):
        self.input_file = input_file
        self.output_file = output_file
        self.tmp_path = tmp_path    
        self.adapter = adapter
        self.sample_name = sample_name
        self.flag = flag
        self.log_file = os.path.join(tmp_path, "log.txt")

    def MeanPhredScore(self, trimmed_quality):
        score_sum = sum(ord(q) - 33 for q in trimmed_quality)
        mean_score = round(score_sum / len(trimmed_quality))
        return mean_score

    def extracted_mirna_with_quality_score(self):
        records = []
        phred_scores = [0] * 100
        with pysam.FastxFile(self.input_file) as f:
            for record in f:
                sequence = record.sequence
                quality = record.quality
                if sequence.count(self.adapter) > 0:
                    trimmed_sequence = sequence.split(self.adapter, 1)[0]
                    trimmed_quality = quality[:len(trimmed_sequence)]
                else:
                    trimmed_sequence = sequence
                    trimmed_quality = quality
                
                if 30 >= len(trimmed_sequence) >= 12:
                    mean_score = self.MeanPhredScore(trimmed_quality)
                    phred_scores[int(mean_score)] += 1
                    if mean_score >= 20:
                        record_id = f"{sample_name}-{len(records)}-score:{mean_score}"
                        records.append((record_id, trimmed_sequence))
        for idx, count in enumerate(phred_scores):
            if count > 0:
                print(f"Score {idx}: {count} sequences")

        with open(self.output_file, "w") as output:
            SeqIO.write((SeqIO.SeqRecord(Seq(sequence), id=record_id, description="") for record_id, sequence in records), output, "fasta")
    
    def modify_fasta_header(self):
        output_records = []
        with gzip.open(self.input_file, "rt") as f:
            for record in SeqIO.parse(f, "fasta"):
                record.id = f"{sample_name}-{len(output_records)}-score:37"
                record.description = ""
                output_records.append(record)
        with open(self.output_file, "w") as f:
            SeqIO.write(output_records, f, "fasta")
    
    def modify_fastq_header(self, tmp_path):
        output_records = []
        with gzip.open(tmp_path + '/' + sample_name + "_trimmed.fq.gz", "rt") as f:
            for record in SeqIO.parse(f, "fastq"):
                record.id = f"{sample_name}-{len(output_records)}-score:37"
                record.description = ""
                output_records.append(record)
        with open(self.output_file, "w") as f:
            SeqIO.write(output_records, f, "fasta")


    def data_process(self):
        log_file = open(self.log_file, "w")
        print(f"[{datetime.datetime.now()}] Start to trim the adapter and low quality reads!")
        fq_extensions = [".fastq", ".fq", ".fastq.gz", ".fq.gz"]
        fa_extensions = [".fasta", ".fa", ".fasta.gz", ".fa.gz"]
        if self.input_file.endswith(tuple(fq_extensions)) and self.flag == False:
            self.extracted_mirna_with_quality_score()
        elif self.input_file.endswith(tuple(fa_extensions)) and self.flag == False:
            self.modify_fasta_header()
        elif self.flag == True:
            trim_cmd = "trim_galore --fastqc --fastqc_args '-t 16 --nogroup' --gzip -q 20 --length 12 --max_length 30 --trim-n --basename {} --no_report_file  -j 8 -a {} -o {} {}".format(self.sample_name, self.adapter, self.tmp_path, self.input_file)
            subprocess.run(trim_cmd, check=True, shell=True, stdin = log_file, stderr= log_file)
            self.modify_fastq_header(tmp_path)
        else:
            print("Please check your input file format!")
            exit(1)
        log_file.close()

if __name__ == "__main__":
    input_file = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/1_rawdata/h-PB1_231215N_S36_L002_R1_001.fastq.gz"
    output_file = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/h-PB1_231215N_S36_L002_R1_001_trimmed.fasta"
    tmp_path = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming"
    adapter = "AACTGTAGGCACCATCAAT"
    sample_name = os.path.basename(input_file).split(".")[0].replace("-", "_")
    tag = input_file.split(".")[-1]
    flag = True
    
    preprocess = Preprocess(input_file, output_file, tmp_path, adapter, sample_name, flag)
    preprocess.data_process()
    


[2024-01-14 23:11:33.221862] Start to trim the adapter and low quality reads!
Analysis complete for h_PB1_231215N_S36_L002_R1_001_trimmed.fq.gz


In [4]:
%%bash
head /bios-store1/chenyc/Project/TMM/Project_tianmiaomiao_231201N/3_tailing_trimming/2_cleandata/EF-19_231201N_S2_L003_R1_001.fasta
echo "--------------"
head /bios-store1/chenyc/scripts/Tailing_Trimming/test/test.fasta
echo "--------------"
head  /bios-store1/chenyc/scripts/Tailing_Trimming/test/test_ef.fasta
echo "--------------"
wc -l  /bios-store1/chenyc/Project/TMM/Project_tianmiaomiao_231201N/3_tailing_trimming/2_cleandata/EF-19_231201N_S2_L003_R1_001.fasta
echo "--------------"
wc -l /bios-store1/chenyc/scripts/Tailing_Trimming/test/test.fasta
echo "--------------"
wc -l /bios-store1/chenyc/scripts/Tailing_Trimming/test/test_ef.fasta

>EF-19_231201N_S2_L003_R1_001-0-score:32
NAATACCGGGCTC
>EF-19_231201N_S2_L003_R1_001-1-score:35
NGTGTTTTCTAGTGTAGGATT
>EF-19_231201N_S2_L003_R1_001-2-score:34
NATAGCTTCGCAG
>EF-19_231201N_S2_L003_R1_001-4-score:35
NTTGGATTGAAGGGAGCTCTTC
>EF-19_231201N_S2_L003_R1_001-5-score:35
NATGAAAACGAAGAGAATCATAA
--------------
>EF-19_231201N_S2_L003_R1_001-0-score:32
NAATACCGGGCTC
>EF-19_231201N_S2_L003_R1_001-1-score:35
NGTGTTTTCTAGTGTAGGATT
>EF-19_231201N_S2_L003_R1_001-2-score:34
NATAGCTTCGCAG
>EF-19_231201N_S2_L003_R1_001-3-score:35
NTTGGATTGAAGGGAGCTCTTC
>EF-19_231201N_S2_L003_R1_001-4-score:35
NATGAAAACGAAGAGAATCATAA
--------------
>EF-19_231201N_S2_L003_R1_001-0-score:37
AATACCGGGCTC
>EF-19_231201N_S2_L003_R1_001-1-score:37
GTGTTTTCTAGTGTAGGATT
>EF-19_231201N_S2_L003_R1_001-2-score:37
ATAGCTTCGCAG
>EF-19_231201N_S2_L003_R1_001-3-score:37
TTGGATTGAAGGGAGCTCTTC
>EF-19_231201N_S2_L003_R1_001-4-score:37
ATGAAAACGAAGAGAATCATAA
--------------


25262310 /bios-store1/chenyc/Project/TMM/Project_tianmiaomiao_231201N/3_tailing_trimming/2_cleandata/EF-19_231201N_S2_L003_R1_001.fasta
--------------
25230746 /bios-store1/chenyc/scripts/Tailing_Trimming/test/test.fasta
--------------
25517692 /bios-store1/chenyc/scripts/Tailing_Trimming/test/test_ef.fasta


In [27]:
class Remapping:
    def __init__(self, output_file, tmp_path, reference, trsnorna, sample_name):
        self.output_file = output_file
        self.tmp_path = tmp_path
        self.reference = reference
        self.trsnoRNA = trsnorna
        self.sample_name = sample_name
        self.unmapped2genome = os.path.join(tmp_path, "unmapped-to-genome.fasta")
        self.unmapped2trsnorna = os.path.join(tmp_path, "clean.fasta")
        self.log_file = os.path.join(tmp_path, "log_remapping.txt")

    def reformatting_fasta(self):
        print(f"[{datetime.datetime.now()}] Reformatting fasta header")
        output_records = []
        with open(self.output_file, "r") as f:
            for record in SeqIO.parse(f, "fasta"):
                record.id = record.id.replace("-score:37", f"-{record.seq}--0")
                record.description = ""
                output_records.append(record)
        with open(self.unmapped2genome, "w") as f:
            SeqIO.write(output_records, f, "fasta")

    def remapping(self):
        log_file = open(self.log_file, "w")
        print(f"Mapping whole reads to trsnoRNA and unmapped reads are saved to {self.tmp_path}/clean.fasta")
        mapping_cmd = "bowtie -p 24 -v 0 -S -a -x {} -f {} --un {}/clean.fasta {}/map2trsnoRNA.sam".format(self.trsnoRNA, self.unmapped2genome, self.tmp_path, self.tmp_path)
        subprocess.run(mapping_cmd, check=True, shell=True, stderr=log_file, stdout=log_file)
        print(f"Mapping whole reads to hairpin20 and unmapped reads are saved to {self.tmp_path}/unmapped-0.fasta")
        mapping_cmd = "bowtie -p 24 -v 0 -S -a -x {} -f {} --un {}/unmapped-0.fasta {}/mapped-0.sam".format(self.reference, self.unmapped2trsnorna, self.tmp_path, self.tmp_path)
        subprocess.run(mapping_cmd, check=True, shell=True, stderr=log_file, stdout=log_file)

        for i in range(1, 11):
            k = i - 1
            output_records = []
            print(f"Trimming {i} bp for each unmapped reads")
            with open(f"{self.tmp_path}/unmapped-{k}.fasta", "r") as in_file:
                for record in SeqIO.parse(in_file, "fasta"):
                    record.seq = record.seq[:-1]
                    if len(record.seq) >= 12:
                        record.id = record.id.split("--")[0] + f"--{i}"
                        output_records.append(record)

            with open(f"{self.tmp_path}/trimmed-{i}.fasta", "w") as out_file:
                SeqIO.write(output_records, out_file, "fasta")
            
            print(f"Mapping trimmed {i} bp reads to hairpin20 and unmapped reads are saved to {tmp_path}/unmapped-{i}.fasta")
            remapping_cmd = "bowtie -p 24 -v 0 -S -a --norc --no-unal -x {} -f {}/trimmed-{}.fasta --un {}/unmapped-{}.fasta {}/mapped-{}.sam".format(self.reference, self.tmp_path, i, self.tmp_path, i, self.tmp_path, i)
            subprocess.run(remapping_cmd, check=True, shell=True, stderr=log_file, stdout=log_file)
        log_file.close()
        

    def merge_result(self):
        for i in range(0, 11):
            print(f"Processing: {self.tmp_path}/mapped-{i}.sam")
            with pysam.AlignmentFile(f"{self.tmp_path}/mapped-{i}.sam", "r", threads=24) as in_file:
                with pysam.AlignmentFile(f"{self.tmp_path}/mapped-{i}.bam", "wb", template=in_file, threads=24) as out_file:
                    for record in in_file:
                        record.query_sequence = record.query_name.split("-")[-3]
                        record.template_length = len(record.query_sequence)
                        record.cigartuples = [(0, len(record.query_sequence))]
                        trim_nu = record.query_sequence[-int(record.query_name.split("--")[1]):]
                        record.set_tag("TM", trim_nu)
                        out_file.write(record)
            pysam.sort("-@", "24", "-o", f"{self.tmp_path}/mapped-{i}.sorted.bam", f"{self.tmp_path}/mapped-{i}.bam")
            pysam.index(f"{self.tmp_path}/mapped-{i}.sorted.bam")
            os.remove(f"{self.tmp_path}/mapped-{i}.sam")
        print(f"[{datetime.datetime.now()}] Merging bam files")
        bam_files = glob.glob(f"{self.tmp_path}/mapped-*.sorted.bam")
        if len(bam_files) > 0:
            pysam.merge("-@", "24", "-f", f"{self.tmp_path}/merged-alignment-sorted.sam", *bam_files)
        else:
            print("No bam files found for merging")
    
    def data_process(self):
        print(f"[{datetime.datetime.now()}] Start to remapping the reads!")
        self.reformatting_fasta()
        self.remapping()
        self.merge_result()
        print(f"[{datetime.datetime.now()}] Remapping finished!")

if __name__ == "__main__":
    reference = "/bios-store1/chenyc/Reference_Source/Arabidopsis_Reference/ath_hairpin_bowtie_index/hairpin"
    trsnoRNA = "/bios-store1/chenyc/Reference_Source/Arabidopsis_Reference/ath_trsnoRNA_bowtie_index/trsnoRNA"
    input_file = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/1_rawdata/h-PB1_231215N_S36_L002_R1_001.fastq.gz"
    output_file = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/h-PB1_231215N_S36_L002_R1_001_trimmed.fasta"
    tmp_path = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming"
    adapter = "AACTGTAGGCAC"
    sample_name = os.path.basename(input_file).split(".")[0].replace("-", "_")
    tag = input_file.split(".")[-1]
    flag = True
    
    remapping = Remapping(output_file, tmp_path, reference, trsnoRNA, sample_name)
    remapping.data_process()


[2024-01-14 23:36:53.860786] Start to remapping the reads!
[2024-01-14 23:36:53.860834] Reformatting fasta header
Mapping whole reads to trsnoRNA and unmapped reads are saved to /bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/clean.fasta
Mapping whole reads to hairpin20 and unmapped reads are saved to /bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/unmapped-0.fasta
Trimming 1 bp for each unmapped reads
Mapping trimmed 1 bp reads to hairpin20 and unmapped reads are saved to /bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/unmapped-1.fasta
Trimming 2 bp for each unmapped reads
Mapping trimmed 2 bp reads to hairpin20 and unmapped reads are saved to /bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/unmapped-2.fasta
Trimming 3 bp for each unmapped reads
Mapping trimmed 3 bp reads to hairpin20 and unmapped reads are saved to /bios-store1/chenyc/Project/TRM-sRNA-seq/h67/2_tailing_trimming/unmapped-3.fasta
Trimming 4 bp for each unma

In [ ]:
OUTDIR = "/bios-store1/chenyc/Project/TRM-sRNA-seq/h67/3_remapping"
PROFILE_PATH = os.path.join(OUTDIR, "4_163.results")
SUMMARY_PATH = os.path.join(OUTDIR, "5_GMC_analysis")
class Profile_maker:
    def __init__(self, meta_file, input_file, output_path, sample_name):
        self.metafile = meta_file
        self.script = os.path.join(os.path.dirname(__file__), "4_163-profile.pl")
        self.output_path = output_path
        self.input_file = input_file # remapping result: merged-alignment-sorted.sam
        self.output_file = os.path.join(self.output_path, sample_name + ".txt")
        
    def dyc_163_profile(self):
        print(f"[{datetime.datetime.now()}] Start to generate the miRNA profile!")
        with open(self.output_file, "w") as f:
            profile_cmd = f"perl self.script {self.metafile} {self.input_file}"
            subprocess.run(profile_cmd, check=True, shell=True, stdout=f)
        print(f"[{datetime.datetime.now()}] Profile generated!")

    def split_163_file(self):
        print(f"[{datetime.datetime.now()}] Start to split the 163 profile file!")
        with open(self.output_file, "r") as f:
            lines = f.readlines()
        matrix = [line.strip().split('\t') for line in lines]
        groups = [matrix[i:i+12] for i in range(0, len(matrix), 12)]
        for group in groups:
            last_line = group.pop()
            newgroup = [row[3:] for row in group[3:]]
            newgroup.insert(0, last_line) 
            group.insert(0, last_line) 
            new_filename = newgroup[0][0].split()[0].replace("*", "_star") + ".txt"
            file = os.path.join(self.output_path, sample_name, new_filename)
            with open(file, "w") as f:
                f.write('\t\n'.join(['\t'.join(row) for row in group]) + '\t\n')

    def martix_summary(self):
        n_lines = len(open(self.output_file, 'r').readlines())
        n_groups = int(n_lines/12)
        res = []
        for i in range(n_groups):
            temp_res = []
            matrix_data = np.loadtxt(self.output_file, skiprows=i*12, max_rows=11)
            tag_data = np.loadtxt(self.output_file, skiprows=i*12+11, max_rows=1, dtype=str)[0]
            temp_res.append(tag_data)
            sum_all = np.sum(matrix_data)
            temp_res.append(sum_all)
            temp_res.append(str(sum_all - np.sum(matrix_data[:,10])))
            temp_res.append(str(sum_all - np.sum(matrix_data[10,:])))
            # for j in range(10):
            for j in range(9,-1,-1):
                temp_res.append(str(np.sum(matrix_data[j,:])))
            res.append(temp_res)

        res = np.array(res)
        np.savetxt(os.path.join(self.output_path, sample_name + ".summary.txt"), res, fmt="%s", delimiter = "\t",
                    header = "ID\tSUM\ttrimming\ttailling\ttail_1\ttail_2\ttail_3\ttail_4\ttail_5\ttail_6\ttail_7\ttail_8\ttail_9\ttail_10")

class Profile_summary:
    def __init__(self, meta_file, input_file, output_path, sample_name):
        self.metafile = meta_file
        self.sample_name = sample_name
        self.input_file = input_file # remapping result: merged-alignment-sorted.sam
        self.output_path = output_path

    def get_mirna_5GMC_reads(self):
        id = []
        site = []
        with open(self.metafile, mode='r') as f:
            data = csv.reader(f, delimiter='\t')
            for d in data:
                id.append(d[0])
                site.append(d[2])
        f.close()

        out = []
        with open(self.input_file, mode='r') as f:
            data = csv.reader(f, delimiter='\t')
            for d in data:
                for i, s in zip(id, site):
                    if i.startswith(d[2]) and d[3] == s:
                        d[2] = i
                        if d[2] == '16':
                            d[9] = reverse_complement(d[9])
                        new_line = '\t'.join([d[0], d[2], d[3], d[9]]) + '\n'
                        out.append(new_line)
                        break
        f.close()
        
        output_file = os.path.join(self.output_path, sample_name + ".5GMC")
        header = "RNAME\tID\tStart\tSequence\n"
        out.insert(0, header)
        with open(output_file, 'w') as f:
            f.writelines(out)
        return output_file

    def get_mirna_5GMC_sequence(self, output_file):
        df = pd.read_csv(output_file, sep='\t')
        groups = df.groupby(df.columns[1])
        for name, group in groups:
            filename = os.path.join(self.output_path, "doc_seqlogo", sample_name, name.replace("*", "_star") + ".txt")
            group.iloc[:, 3].apply(lambda x: x.replace('T', 'U')).to_csv(filename, sep='\t', index=False, header=False)

        with open(self.metafile, 'r') as f:
            for line in f:
                file_name = line.strip().split("\t")[0].replace("*","_star")
                file_path = os.path.join(self.output_path, file_name + ".txt")
                if not os.path.exists(file_path):
                    with open(file_path, 'w') as f:
                        f.write('N'*30)
    
    